# Processing and Analysis of Raster Data

In the first part of our lesson, we reviewed the basics of raster files and explored how to read and work with both single-band and multiband raster files, as well as retrieve raster images via WMS. Now, in the second part of the lesson, we will focus on how to process and analyze raster data. Specifically, we will cover:

- Merging multiple raster files to create a `raster mosaic`
- Clipping rasters using a polygon
- Reclassifying raster data
- Performing `slope` analysis

For this part of the lesson, we will continue using the elevation model data provided by the [National Land Survey of Finland](https://www.maanmittauslaitos.fi/en/maps-and-spatial-data/datasets-and-interfaces/product-descriptions/elevation-model-2-m). This time, we will work with all four raster tiles (L4133A, L4133B, L4133C, L4133D), which cover parts of Helsinki's city center. These tiles have already been downloaded to the `data` directory and will be used in this lesson.

Now let's read all four raster tiles and visualize them using subplots.   

In [2]:
import rioxarray
import matplotlib.pyplot as plt

# Load the rasters and reproject them to EPSG:3067
raster1 = rioxarray.open_rasterio('data/L4133A.tif').rio.reproject("EPSG:3067")
raster2 = rioxarray.open_rasterio('data/L4133B.tif').rio.reproject("EPSG:3067")
raster3 = rioxarray.open_rasterio('data/L4133C.tif').rio.reproject("EPSG:3067")
raster4 = rioxarray.open_rasterio('data/L4133D.tif').rio.reproject("EPSG:3067")

# Create a subplot with 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot each raster in a separate subplot
raster1.plot(ax=axes[0, 0], cmap='viridis')
axes[0, 0].set_title('Raster 1 (L4133A)')

raster2.plot(ax=axes[0, 1], cmap='viridis')
axes[0, 1].set_title('Raster 2 (L4133B)')

raster3.plot(ax=axes[1, 0], cmap='viridis')
axes[1, 0].set_title('Raster 3 (L4133C)')

raster4.plot(ax=axes[1, 1], cmap='viridis')
axes[1, 1].set_title('Raster 4 (L4133D)')

# Adjust layout
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'rioxarray'

<div style="border-left: 4px solid #f39c12; background-color: #fdf6e3; padding: 15px; margin: 20px 0;">
    <strong>Note:</strong> We are again using <code>matplotlib</code> for visualization our raster datasets. Everything we’ve previously learned about visualization with <code>matplotlib</code> and styling still applies here. We have extensively used <code>matplotlib</code> during the 
    <a href="https://geo-python-site.readthedocs.io/en/latest/lessons/L7/python-plotting.html" target="_blank">GeoPython Lesson 7</a> and throughout this course, especially in 
    <a href="https://autogis-site.readthedocs.io/en/latest/lessons/lesson-5/static-maps.html" target="_blank">Lesson 5</a> where we focused on creating static maps.
</div>


As you can see, the four raster tiles cover adjacent areas in the Helsinki city center. However, they are still separate files! In the next step, we will merge these tiles into a single raster mosaic, creating a seamless representation that covers the entire central area of Helsinki.

### Create raster mosaic 

Raster files are usualy large in size, therefor it is quite common for providers to publish them in smaller pieces or tiles. While this makes it easier to transfer the data, it may not be so practical when it comes to the actual analysis. For example, as seen above, the elevation model from center of Helsinki is divided into 4 separate raster files. 

Now we want to merge multiple raster files together and create a raster mosaic. This can be done with the `merge_arrays()` -function in `rioxarray`.

In our case, the raster files are coming from the same source, and are actually pieces of a larger data. If that was not the case, it would have been a good idea to start by aligning our raster tiles, ensuring that they can be combined without alignment problems. The `Align` function helps align your raster files in terms of resolution, projection, and spatial extent.

```
import xarray as xr
# Ensure all rasters have the same CRS
assert raster1.rio.crs == raster2.rio.crs == raster3.rio.crs == raster4.rio.crs, "Rasters have different CRS"

# Align the rasters to the same resolution and spatial extent
raster1, raster2 = xr.align(raster1, raster2)
raster3, raster4 = xr.align(raster3, raster4)
```

Now let's merge our data:

In [ ]:
from rioxarray.merge import merge_arrays

# Merge the four raster data into one
mosaic_merged = merge_arrays([raster1, raster2, raster3, raster4])

# Save the merged raster (optional)
# mosaic_merged.rio.to_raster('merged_raster.tif')

# Plot the mosaic raster
mosaic_merged.plot()

### Clipping raster using a polygon

Clipping a raster using vector data (i.e., a polygon) is another common operation with raster data. In this part of lesson, we use wfs to get the administrative boundaries in Helsinki area. 

In [ ]:
from owslib.wfs import WebFeatureService
import geopandas as gpd

# Define the bounding box for Helsinki (EPSG:3879)
bbox_helsinki = "25400000,6670000,25500000,6680000"

# Fetch the Paavo data limited to the Helsinki area using WFS
paavo_data_helsinki = gpd.read_file(
    "https://geo.stat.fi/geoserver/wfs"
    "?service=wfs"
    "&version=2.0.0"
    "&request=GetFeature"
    "&typeName=postialue:pno"  # Adjust to the correct layer if needed
    "&srsName=EPSG:3879"
    f"&bbox={bbox_helsinki},EPSG:3879"
)

# Display the first few rows of the data
paavo_data_helsinki.head()

# Save to a file if needed
# paavo_data_helsinki.to_file("paavo_data_helsinki.geojson", driver="GeoJSON")


Now we can use the contents of the column *posti_alue* to select a ppostal area of our choice. Let's try Kamppi; we know that the postal code for Kamppi is `00100`.

In [ ]:
# Select Kamppi polygon based on postal code
kamppi = paavo_data_helsinki[paavo_data_helsinki['posti_alue'] == '00100']

# Transfrom the polygon to the same CRS as our raster data
kamppi = kamppi.to_crs(mosaic_merged.rio.crs)

kamppi.plot()

> **Note:**
> Similar to other map overlay analyses, it is essential to ensure that the CRS (Coordinate Reference System) of the raster files and the clipping feature are matching.

Once again, let's make sure that the CRS are matching and then let's proceed with the clipping. 

In [ ]:
# Double check that the CRS match
assert raster1.rio.crs == kamppi.crs , "CRS Mismatch"

# Clip the raster using the Kamppi polygon
clipped_mosaic = mosaic_merged.rio.clip(kamppi.geometry, kamppi.crs)

# Save the clipped raster (optional)
# clipped_mosaic.rio.to_raster("clipped_mosaic_kamppi.tif")

clipped_mosaic.plot()

In [ ]:
nodata_value = clipped_mosaic.rio.nodata
# Mask the NoData values
clipped_raster = clipped_mosaic.where(clipped_mosaic != nodata_value)
clipped_raster.plot()

## Raster reclassification

**Raster reclassification** is another common procedure with raster dataset. Raster reclassification is the process of assigning new values to the pixels of a raster dataset based on their existing values. This is often used to simplify or categorize continuous data, such as elevation or land cover, into distinct classes. For example, in an elevation map, reclassification can group different elevation ranges into categories like "low", "medium", and "high" altitude, making it easier to analyze or visualize the data for specific applications such as hazard assessment or land management.

let's first have a look at the range of values (altitudes) in our raster data:

In [ ]:
# Check the data range
print(clipped_raster.min().values, raster.max().values)

Now we want to reclassify our data using two approaches. First, let's do it manually using the `numpy` library. **[NumPy](https://numpy.org/doc/)** is a powerful Python library used for numerical and scientific computing. It provides support for large, multi-dimensional arrays and matrices, along with a wide collection of mathematical functions to operate on these arrays efficiently. So basically here, we are treating our pixel values as an `Array` and do the calculations accordingly. 

> **Note:**
> An **array** (such as those created by NumPy) is a multi-dimensional container for data, but it lacks metadata like coordinate labels or attributes. In contrast, an **xarray.DataArray** enhances the array by adding labeled dimensions, coordinates, and attributes, making it easier to work with multi-dimensional data, especially in geospatial and time-series contexts.


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Define the bins and new class values
# 'bins' specifies the thresholds for reclassification: 
# - Values less than 0 will be classified as "Low"
# - Values between 0 and 10 will be classified as "Mid"
# - Values greater than 10 will be classified as "High"
bins = [-50, 0, 10, np.inf]  # Bins for elevation: <0, 0-10, 10+

# 'new_values' contains the labels or categories corresponding to the bin ranges
new_values = [1, 2, 3]  # Corresponding new class values

# Apply the reclassification
# 'np.digitize' assigns each value in 'raster' to a bin defined in 'bins'
# 'right=True' means each bin includes its right edge, e.g., 0 is in the "Mid" category
# This returns an array where each pixel is assigned a class index (1 for Low, 2 for Mid, etc.)
reclassified_raster = np.digitize(raster, bins, right=True)

# Convert the reclassified array to an xarray.DataArray
# 'dims' preserves the original raster dimensions (e.g., x, y coordinates)
# 'coords' retains the spatial coordinate information from the original raster
# 'attrs' keeps any metadata associated with the raster (e.g., CRS, description)
reclassified_raster = xr.DataArray(
    reclassified_raster,
    dims=raster.dims,  # Keep the same dimensions as the original raster
    coords=raster.coords,  # Retain the spatial coordinates (x, y)
    attrs=raster.attrs  # Preserve the original attributes (e.g., CRS, metadata)
)

plt.figure(figsize=(8, 6))
cmap = plt.matplotlib.colors.ListedColormap(['yellow', 'red', 'brown'])
plt.colorbar(ticks[1, 2, 3], label= "Elevation class")
plt.title("Classified elevation")
reclassified_raster.plot()

> **Note:**
> When using `np.digitize()`, the function returns indices of the bins, starting from `1`. This means the output will not directly contain the desired reclassified values. To map the bin indices to the actual values, you need to create a mapping, such as using a list comprehension to convert the indices into the specified new values. Don't forget to subtract `1` from the indices because Python lists are zero-indexed, while `np.digitize()` starts indexing at `1`.


In [ ]:
# Define the bins and new class values
bins = [-50, 0, 10, np.inf]  # Bins for elevation: <0, 0-10, 10+
new_values = [4, 7, 10]  # Corresponding new class values

# Apply the reclassification
# 'np.digitize' assigns each value in 'raster' to a bin defined in 'bins'
# This will give indices: 1 for the first bin, 2 for the second bin, etc.
reclassified_indices = np.digitize(raster, bins, right=True)

# Create an empty array with the same shape as the reclassified indices
reclassified_raster = np.empty_like(reclassified_indices, dtype=int)

# Iterate over the array using np.nditer and map the indices to the new values
for idx, value in np.ndenumerate(reclassified_indices):
    reclassified_raster[idx] = new_values[value - 1]  # Map the index to the corresponding class value

# Convert the reclassified array to an xarray.DataArray
reclassified_raster = xr.DataArray(
    reclassified_raster,
    dims=raster.dims, 
    coords=raster.coords,  
    attrs=raster.attrs  
)

# Now reclassified_raster will contain the values 4, 7, and 10 based on the bins.
reclassified_raster.plot()

We can use the `mapclassify` library which we learned about in Lesson 4, to reclassify our raster. Let's Try *NaturalBreaks* method here.  

In [ ]:
import mapclassify

# Flatten the raster data
raster_data_flattened = clipped_raster.values.flatten()

# Filter out NoData values
raster_data_flattened = raster_data_flattened[~np.isnan(raster_data_flattened)]

# Apply Natural Breaks classification with a specified number of classes (e.g., 5)
natural_breaks = mapclassify.NaturalBreaks(raster_data_flattened, k=7)

# Get the classified bins (breaks)
breaks = natural_breaks.bins

# Use np.digitize to classify the raster values according to the natural breaks
reclassified_raster = np.digitize(raster.values, bins=breaks)


# Convert the reclassified array to DataArray
reclassified_raster_nb = xr.DataArray(
    reclassified_raster,
    dims=raster.dims,
    coords=raster.coords,
    attrs=raster.attrs
)
reclassified_raster_nb.plot()

## Slope analysis

**Slope analysis** is a key terrain analysis technique used in GIS and spatial analysis to measure the steepness or incline of the terrain at any given point. It is calculated by examining the rate of change in elevation between neighboring pixels in a digital elevation model (DEM). Slope analysis helps identify areas with steep gradients, which is useful in applications like land-use planning, erosion risk assessment, hydrological modeling, and infrastructure development. The slope is typically expressed in degrees or as a percentage.

Of course, we can perform slope analysis anywhere, including on the Helsinki city center raster we worked with earlier. However, for this course's purpose, it will be more interesting to calculate it for an area with more elevation variation. Since Finland is generally quite flat, we need to look further north to find terrain that is more elevation-wise interesting for slope analysis. For this purpose, we are heading to [Ylläs](https://en.wikipedia.org/wiki/Yll%C3%A4s) in Lapland, located in the northern part of Finland. Ylläs is a fell, or mountainous hill, and offers more diverse elevation changes, making it ideal for our slope analysis!

<div style="border-left: 4px solid #3498db; background-color: #f0f8ff; padding: 15px; margin: 20px 0;">
    <h3>About Ylläs</h3>
    <p>Ylläs is one of the highest fells in Finland, standing at 718 meters. It is located in the Pallas-Yllästunturi National Park and is a popular destination for outdoor activities such as hiking and skiing. Ylläs is known for its stunning natural landscapes, including vast forests, pristine lakes, and dramatic fells that rise above the surrounding terrain.</p>
    <img src="fig/yllas.jpg" alt="Raster Data Representation" style="max-width:100%; height:auto;">
    <p><em>Image Source: <a href="https://en.wikipedia.org/wiki/Yll%C3%A4s" target="_blank">Wikipedia - Ylläs</a> - <a href="https://www.metsa.fi/en/responsible-business/kiinteistokauppa-eng/business-sites/spectacular-yllas/" target="_blank">Metsähallitus</a></em></p>
</div>

The raster we will be working with is from the same source as the previous ones, provided by the [National Land Survey of Finland](https://www.maanmittauslaitos.fi/en/maps-and-spatial-data/datasets-and-interfaces/product-descriptions/elevation-model-2-m), but from a different tile (`U4234`). We will calculate the slope using our clipped elevation raster. The slope at each point is derived by determining the rate of change in elevation between neighboring cells in the raster. This is done by:

1. **Gradient Calculation**: We use the elevation differences between adjacent cells in both the x (horizontal) and y (vertical) directions. The gradient represents how quickly elevation changes in these directions.
   
2. **Slope Formula**: The slope is calculated by combining these gradients using the Pythagorean theorem:
$$
\text{slope} = \sqrt{\left(\frac{\Delta z}{\Delta x}\right)^2 + \left(\frac{\Delta z}{\Delta y}\right)^2}
$$

where $ \Delta z $ is the change in elevation, and $ \Delta x $ and $ \Delta y $ are the distances between the cells in the x and y directions.


3. **Final Slope Values**: The result is often expressed in degrees or as a percentage, with steeper areas showing higher slope values. We use degrees here. 

In [ ]:
yllas_dem = rioxarray.open_rasterio('data/U4234A.tif')
yllas_dem.plot()

In [ ]:
# Get the pixel resolution (assuming square pixels)
xres = yllas_dem.rio.resolution()[0]  # Resolution in x direction (longitude)
yres = yllas_dem.rio.resolution()[1]  # Resolution in y direction (latitude)

# Calculate gradients in the x and y directions
dzdx = yllas_dem.differentiate(coord='x') / xres  # Gradient in the x direction
dzdy = yllas_dem.differentiate(coord='y') / yres  # Gradient in the y direction

# Calculate the slope (in degrees)
slope = np.sqrt(dzdx**2 + dzdy**2)
slope = np.arctan(slope) * (180 / np.pi)

# Update the attributes to reflect that this is a slope raster
slope.attrs['long_name'] = 'Slope'
slope.attrs['units'] = 'degrees'

Now let's plot our Slope raster. 

In [ ]:
# Plot the slope raster
plt.figure(figsize=(10, 8))
slope.plot(cmap='terrain', add_colorbar=True)
plt.title("Slope Map (Degrees)")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

### Raster-to-Raster Calculations

Raster-to-raster calculations involve performing mathematical or logical operations on two or more raster datasets to derive new information. Each raster grid cell is processed based on corresponding cells from the input rasters. This method is commonly used in geospatial analysis to perform tasks like:

- **Combining terrain attributes**: Slope, aspect, and DEM can be used to calculate terrain stability or hydrological indices.
- **Environmental modeling**: Calculating indices such as vegetation health (NDVI) using different raster bands.
- **Suitability analysis**: Combining different factors such as elevation and slope to determine land use or habitat suitability.

**Example: Slope Stability Index (SSI)**
*SSI* is a basic measure to assess the potential stability of a terrain. It combines both the slope steepness and elevation of the terrain, providing an index that indicates the likelihood of a slope to remain stable. Steeper slopes and higher elevations tend to be more prone to instability, which can lead to phenomena such as landslides, rockfalls, or soil erosion.

The Slope Stability Index can be calculated as:

$$
\text{SSI} = \frac{1}{\text{Slope} \times \text{Elevation}}
$$

- **Slope**: The steepness of the terrain, typically measured in degrees or percentage.
- **Elevation (DEM)**: The height of the terrain above a reference point (usually sea level).
- **SSI**: The lower the value of the SSI, the higher the instability risk. Conversely, higher values indicate more stable terrain.


Now let's write this in Python:

In [ ]:
import numpy as np
import rioxarray as rxr

# Read the first band (assuming single-band rasters)
slope = slope[0]
dem = yllas_dem[0]

# Avoid division by zero by setting zero or negative DEM values to NaN
dem = yllas_dem.where(dem > 0)

# Calculate the Slope Stability Index (SSI)
ssi = 1 / (slope * dem)

# Handle any infinities resulting from division by zero (set them to NaN)
ssi = ssi.where(np.isfinite(ssi), np.nan)

# Update SSI attributes to match input slope raster
ssi.rio.write_crs(slope.rio.crs)
ssi.rio.write_nodata(np.nan, inplace=True)

# Save the result as a GeoTIFF
ssi.plot()